# ANDES interface with pandapower

This example (1) shows how to convert an ANDES system (ssa) to a pandapower network (ssp) (2) benchmarks the powerflow results.

In [1]:
import andes
from andes.utils.paths import get_case
from andes.io.pandapower import to_pandapower
andes.config_logger(stream_level=20)

import pandapower as pp
import numpy as np
import pandas as pd

from math import pi

## Load case

In [2]:
ssa = andes.load(get_case('ieee14/ieee14_ieeet1.xlsx'),
                 setup=False,
                 no_output=True,
                 default_config=True)
ssa.setup()

Working directory: "/Users/jinningwang/Documents/work/andes/examples"
Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "/Users/jinningwang/Documents/work/andes/andes/cases/ieee14/ieee14_ieeet1.xlsx"...
Input file parsed in 0.1511 seconds.
System internal structure set up in 0.0377 seconds.


True

In [3]:
ssa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  A total of 1 island(s) detected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0038 seconds.
0: |F(x)| = 0.5605182134
1: |F(x)| = 0.006202200332
2: |F(x)| = 5.819382825e-06
3: |F(x)| = 6.967745825e-12
Converged in 4 iterations in 0.0066 seconds.


True

## Convert to ssp

In [4]:
ssp = to_pandapower(ssa)

In [5]:
ssp

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (5 elements)
   - shunt (2 elements)
   - line (16 elements)
   - trafo (4 elements)

## Comapre Power Flow

In [6]:
# ssa
ssa_res_gen = pd.DataFrame(columns=['name', 'p_mw', 'q_mvar', 'va_degree', 'vm_pu'])
ssa_res_gen['name'] = ssa.PV.as_df()['name']
ssa_res_gen['p_mw'] = ssa.PV.p.v * ssa.config.mva
ssa_res_gen['q_mvar'] = ssa.PV.q.v * ssa.config.mva
ssa_res_gen['va_degree'] = ssa.PV.a.v * 180 / pi
ssa_res_gen['vm_pu'] = ssa.PV.v.v
ssa_res_slack = pd.DataFrame([[ssa.Slack.name.v[0], ssa.Slack.p.v[0] * ssa.config.mva,
                              ssa.Slack.q.v[0] * ssa.config.mva, ssa.Slack.a.v[0] * 180 / pi,
                              ssa.Slack.v.v[0]]],
                             columns=ssa_res_gen.columns,
                             )
ssa_res_gen = pd.concat([ssa_res_gen, ssa_res_slack]).reset_index(drop=True)

# ssp
pp.runpp(ssp)
ssp_res_gen = pd.concat([ssp.gen['name'], ssp.res_gen], axis=1)

pf_gen = pd.concat([ssa_res_gen, ssp_res_gen], axis=1)

pf_bus = ssa.Bus.as_df()[["name"]].copy()
# ssa
pf_bus['v_ad'] = ssa.Bus.v.v
pf_bus['a_ad'] = ssa.Bus.a.v * 180 / pi

# ssp
pf_bus['v_pp'] = ssp.res_bus['vm_pu']
pf_bus['a_pp'] = ssp.res_bus['va_degree']

## Generation

In [7]:
pf_gen.round(4)

,name,p_mw,q_mvar,va_degree,vm_pu,name,p_mw,q_mvar,va_degree,vm_pu
0,2,40.0000,30.4361,-1.7641,1.03,2,40.0000,30.4361,-1.7641,1.03
1,3,40.0000,12.5971,-3.5371,1.01,3,40.0000,12.5971,-3.5371,1.01
2,4,30.0000,20.9866,-6.4527,1.03,4,30.0000,20.9866,-6.4527,1.03
3,5,35.0000,7.3964,-1.5400,1.03,5,35.0000,7.3964,-1.5400,1.03
4,1,81.4272,-21.6171,0.0000,1.03,1,81.4272,-21.6171,0.0000,1.03


## Bus voltage and angle

In [8]:
pf_bus.round(4)

,name,v_ad,a_ad,v_pp,a_pp
uid,,,,,
0,BUS1,1.0300,0.0000,1.0300,0.0000
1,BUS2,1.0300,-1.7641,1.0300,-1.7641
2,BUS3,1.0100,-3.5371,1.0100,-3.5371
3,BUS4,1.0114,-4.4098,1.0114,-4.4098
4,BUS5,1.0173,-3.8430,1.0173,-3.8430
5,BUS6,1.0300,-6.4527,1.0300,-6.4527
6,BUS7,1.0225,-4.8852,1.0225,-4.8852
7,BUS8,1.0300,-1.5400,1.0300,-1.5400
8,BUS9,1.0218,-7.2459,1.0218,-7.2459
